# 1.2 bootstrapping
last updated : 2023-06-25

부트스트래핑과정을 파이썬으로 실습해보는 과정
이자지급주기 1년을 가정하고, 관측되는 ytm의 기간은 1년, 3년, 5년물이며, 각각의 ytm은 1% , 3% , 5%로 가정

### input 정의
- basis : 이자지급주기
- ytm_periods : 관측된 ytm의 연도
- ytm_interests : 관측된 ytm의 이자율

#### 1. ytm 보간법
관측되는 ytm과 이자율을 통해서 이자지급주기마다의 보간법이 적용된 ytm 이자율들을 구합니다.  

In [3]:
import numpy as np

basis = 1 # 이자지급 1년에 1번을 가정합니다. 
ytm_periods = [1,3,5] # 관측기간은 1년, 3년, 5년
ytm_interests = [0.01, 0.03, 0.05] 
bond_principal = 100 # 채권이자원금 100원 가정

In [4]:
# basis 마다 노드 생성
basis_array = np.arange(basis, 5 + basis, basis)

# 출력 : [1 2 3 4 5]


# ytm 보간법
interpolated_ytm_interests = np.interp(
    basis_array, ytm_periods, ytm_interests)

# 출력 : array([0.01, 0.02, 0.03, 0.04, 0.05])

#### 2. spot 이자율 산출
보간법이 적용된 ytm 이자율들을 활용하여, spot 이자율을 산출하는 과정이다.
보다 자세한 내용은 본 책 내용의 xxx를 참고  

In [5]:
spot_rates = [interpolated_ytm_interests[0]] # 가장 첫번째 spot 이자율은 ytm 관측이자율로 고정

for idx, ytm in enumerate(interpolated_ytm_interests[1:]):
    t = (idx + 1) * basis
    pv_interest = 0
    for j in range(idx + 1):

        # 기간별 원금이자 계산
        current_interest = bond_principal * basis * ytm
        exponent = j + 1

        # 기간에 대응하는 Spot rate로 원금이자를 할인하여 이자의 현재가치 계산
        pv_current_interest = current_interest / np.power((1 + spot_rates[j]), exponent)

        # 이자누적
        pv_interest += pv_current_interest

    # 만기원리금 계산 : 원금 + (1 + ytm * basis)
    total_interest_principal = bond_principal * (1 + ytm)

    # 만기원리금 계산 : 원금 + (1 + ytm * basis)
    pv_bond_price = bond_principal - pv_interest

    exponent = idx + 2
    pv_factor = total_interest_principal / pv_bond_price
    spot_rate = (np.power(pv_factor, 1 / exponent) - 1)
    spot_rates.append(spot_rate)


In [6]:
# 결과를 dataframe으로 만들고 엑셀로 저장하기
import pandas as pd
df = pd.DataFrame(spot_rates)
df.to_excel("Ch1.2_bootstrapping_spot_rates.xlsx")

#### 3. forward 이자율 산출
각 기간별 spot 이자율을 산출한 후에 각 시점별 forward 이자율을 산출합니다.

In [8]:
forward_rates = [spot_rates[0]]
for i in range(1, len(spot_rates)):
    t1 = i + 1
    t2 = i
    
    forward_rate = ((np.power((1 + spot_rates[i]), t1) / np.power((1 + spot_rates[i - 1] ), t2)) - 1)
    forward_rates.append(forward_rate)

In [9]:
# 결과를 dataframe으로 만들고 엑셀로 저장하기
import pandas as pd
forward_rates_df = pd.DataFrame(forward_rates)
forward_rates_df.to_excel("Ch1.2_bootstrapping_forward_rates.xlsx")